In [21]:
import gym
import numpy as np

import time

In [22]:
import sys
import os

sys.path.append(os.path.abspath('../'))

del sys, os

In [23]:
import matplotlib.pyplot as plt

In [24]:
# LaTeX rendering in graphs
from distutils.spawn import find_executable
if find_executable('latex'):
    plt.rc('text', usetex=True)

plt.rc('font', family='serif')

# High resolution graphs
%config InlineBackend.figure_format = 'retina'

In [25]:
import torch

In [26]:
%reload_ext autoreload
%autoreload 2

In [27]:
import models.rnn as rnns
import models.mlp as mlps
import control.agents as agents
import control.environments as env

In [28]:
from utils.notifications import Slack

In [29]:
import copy

# Setup

In [30]:
# model = mlps.MLP()
model = rnns.AttentiveRNN(input_dimension=128, hidden_dimension=50, truncate=10)  #, key_dimension=15)

In [31]:
optimiser = torch.optim.Adam(model.parameters(), lr=.001)

In [32]:
agent = agents.DQNAgent(model, optimiser, gamma=.9999, temperature=100, algorithm='qlearning')

In [33]:
environment = env.Environment(
    environment=gym.make('Breakout-ram-v4'), 
    agent=agent,
    verbose=True,
    max_steps=1000,
    slack=Slack()
)

In [15]:
environment.save('saved/test')

In [15]:
from utils.checkpoint import load_environment

In [16]:
load_environment('saved/test').agent.model

AttentiveRNN(
  (activation): ReLU()
  (dropout): Dropout(p=0.1)
  (input_layer): Linear(in_features=128, out_features=50, bias=True)
  (context_layer): Linear(in_features=100, out_features=50, bias=True)
  (key): Linear(in_features=50, out_features=5, bias=True)
  (query): Linear(in_features=50, out_features=5, bias=True)
  (action_layer): Linear(in_features=50, out_features=4, bias=True)
)

# Experiment

In [43]:
environment.exploration_segment(20)

(0.0, 0.0)

In [47]:
environment.batch(100)

In [37]:
environment.reset()
s = environment.state

In [38]:
agent.q(s)

array([-0.06783922, -0.04041   , -0.04206257, -0.1287726 ], dtype=float32)

In [123]:
agent.commit()

In [124]:
agent.q(x)

array([-0.7905634 , -0.24595691, -0.15114474, -0.5289955 ], dtype=float32)

In [31]:
environment.evaluation_episode()

(0.0, 1000)

In [1750]:
state = environment.environment.reset()

agent.eval()

done = False
full_return = 0.

action = np.random.choice(environment.greedy(state))

counter = 0
while not done and counter < 10000:
    
    s, reward, done, i = environment.environment.step(action)

    # If there are ties, we might want to choose between actions at random
    a = np.random.choice(environment.greedy(s))

    # We store the new state and action
    state, action = s, a
    
    full_return = agent.gamma * full_return + reward
    counter += 1

full_return, counter

(0.0, 160)

In [1515]:
state = environment.environment.reset()
action = np.random.choice(environment.greedy(state))

In [1784]:
s, reward, done, i = environment.environment.step(action)

# If there are ties, we might want to choose between actions at random
# a = np.random.choice(4, p=environment.epsilon_greedy(s, .1))
a = environment.environment.action_space.sample()

# We store the new state and action
state, action = s, a

done = done or i['ale.lives'] < 5

done, i, a

(True, {'ale.lives': 4}, 0)

# Training

In [1752]:
environment.run(epochs=2, segments=5, episodes=50)

100%|#################################################################| 5/5 [00:45<00:00,  9.41s/it]


In [23]:
environment.save('../saved/')